In [1]:
from pathlib import Path
import pandas as pd
import csv

# 👉 Change this if you moved the IMDB folder elsewhere
IMDB_DIR = Path(r"C:\Users\abhij\Downloads\IMDB Dataset")

def find_file(root: Path, filename: str) -> Path:
    """Return the first real file named `filename` under root (recursively)."""
    matches = [p for p in root.rglob(filename) if p.is_file()]
    if not matches:
        raise FileNotFoundError(f"Couldn't find {filename} under {root}")
    # If multiple matches, pick the largest (usually the real dataset file)
    return max(matches, key=lambda p: p.stat().st_size)

# Locate the true files (handles the 'folder-with-same-name' situation)
IMDB_TRAIN_PATH = find_file(IMDB_DIR, "labeledTrainData.tsv")
IMDB_TEST_PATH  = find_file(IMDB_DIR, "testData.tsv")
IMDB_UNLAB_PATH = find_file(IMDB_DIR, "unlabeledTrainData.tsv")

print("IMDB train file:", IMDB_TRAIN_PATH)
print("IMDB test file :", IMDB_TEST_PATH)
print("IMDB unlab file:", IMDB_UNLAB_PATH)

# Read TSVs
train_df = pd.read_csv(IMDB_TRAIN_PATH, sep="\t", encoding="utf-8", quoting=csv.QUOTE_NONE)
test_df  = pd.read_csv(IMDB_TEST_PATH,  sep="\t", encoding="utf-8", quoting=csv.QUOTE_NONE)

# Unlabeled is optional; only 'id' and 'review' columns
try:
    unlab_df = pd.read_csv(IMDB_UNLAB_PATH, sep="\t", encoding="utf-8", quoting=csv.QUOTE_NONE)
except Exception as e:
    print("Unlabeled load skipped:", e)
    unlab_df = None

train_df.head(), test_df.head()


C:\Users\abhij\anaconda3\envs\flask\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


IMDB train file: C:\Users\abhij\Downloads\IMDB Dataset\labeledTrainData.tsv
IMDB test file : C:\Users\abhij\Downloads\IMDB Dataset\testData.tsv
IMDB unlab file: C:\Users\abhij\Downloads\IMDB Dataset\unlabeledTrainData.tsv


(         id  sentiment                                             review
 0  "5814_8"          1  "With all this stuff going down at the moment ...
 1  "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ...
 2  "7759_3"          0  "The film starts with a manager (Nicholas Bell...
 3  "3630_4"          0  "It must be assumed that those who praised thi...
 4  "9495_8"          1  "Superbly trashy and wondrously unpretentious ...,
            id                                             review
 0  "12311_10"  "Naturally in a film who's main themes are of ...
 1    "8348_2"  "This movie is a disaster within a disaster fi...
 2    "5828_4"  "All in all, this is a movie for kids. We saw ...
 3    "7186_2"  "Afraid of the Dark left me with the impressio...
 4   "12128_7"  "A very accurate depiction of small time mob l...)

In [2]:
%pip uninstall -y torch torchvision torchaudio
# PyTorch 2.3.1 CPU wheels + matching transformers
%pip install -q --no-cache-dir torch==2.3.1 --index-url https://download.pytorch.org/whl/cpu
%pip install -q --no-cache-dir transformers==4.42.4 accelerate==0.30.1


Found existing installation: torch 2.3.1+cpu
Uninstalling torch-2.3.1+cpu:
  Successfully uninstalled torch-2.3.1+cpu
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch, transformers
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)

# this should now import without error
from transformers import pipeline
print("pipelines import OK")


Torch: 2.3.1+cpu
Transformers: 4.42.4

pipelines import OK


In [4]:
#Imports, helpers, random seeds
import os, random, textwrap, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from tqdm import tqdm

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# LLMs
from transformers import pipeline

# Classic ML utils
from sklearn.datasets import fetch_20newsgroups, load_wine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); tf.random.set_seed(SEED)

def hrule(title=None):
    bar = "=" * 72
    print("\n" + bar)
    if title:
        print(title)
        print(bar)
    else:
        print(bar)


In [5]:
import os

path = r"C:\Users\abhij\Downloads\IMDB Dataset\labeledTrainData.tsv"

print("Is it a file?", os.path.isfile(path))
print("Is it a folder?", os.path.isdir(path))

# List files inside, if it's a folder
if os.path.isdir(path):
    print("Contents:", os.listdir(path))



Is it a file? True
Is it a folder? False


In [6]:
from pathlib import Path
import pandas as pd

# point to the EXTRACTED file, not the compressed folder
IMDB_CSV = Path(r"C:\Users\abhij\Downloads\IMDB Dataset\labeledTrainData.tsv")  
TITANIC_CSV = Path(r"C:\Users\abhij\Downloads\titanic\train.csv")

imdb_df = pd.read_csv(IMDB_CSV, sep="\t")
titanic_df = pd.read_csv(TITANIC_CSV)

print("IMDB shape:", imdb_df.shape)
print("Titanic shape:", titanic_df.shape)
display(imdb_df.head(3))
display(titanic_df.head(3))



IMDB shape: (25000, 3)
Titanic shape: (891, 12)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [7]:
from pathlib import Path
import pandas as pd

# 🔹 Paths to your real datasets
IMDB_CSV = Path(r"C:\Users\abhij\Downloads\IMDB Dataset\labeledTrainData.tsv")
TITANIC_CSV = Path(r"C:\Users\abhij\Downloads\titanic\train.csv")

# 🔹 Check existence
print("IMDB exists:", IMDB_CSV.exists())
print("Titanic exists:", TITANIC_CSV.exists())

# 🔹 Load datasets
imdb_df = pd.read_csv(IMDB_CSV, sep="\t")   # TSV → tab separated
titanic_df = pd.read_csv(TITANIC_CSV)       # CSV → normal comma separated

print("IMDB shape:", imdb_df.shape)
print("Titanic shape:", titanic_df.shape)

# Show sample rows
display(imdb_df.head(3))
display(titanic_df.head(3))


IMDB exists: True
Titanic exists: True
IMDB shape: (25000, 3)
Titanic shape: (891, 12)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [8]:
from transformers import pipeline

# Public, CPU-friendly models (no login/token)
summarizer = pipeline("text2text-generation", model="google/flan-t5-small")
zshot      = pipeline("zero-shot-classification", model="typeform/distilbert-base-uncased-mnli")

def summarize(text, max_new_tokens=80):
    prompt = "summarize: " + text.strip().replace("\n"," ")
    out = summarizer(prompt, max_new_tokens=max_new_tokens, do_sample=False)
    return out[0]["generated_text"]

def zero_shot_label(text, candidate_labels):
    res = zshot(text, candidate_labels=candidate_labels, multi_label=False)
    return res["labels"][0], float(res["scores"][0])



The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [9]:
# 🔹 Select one IMDB review for LLM summary + zero-shot demo
PT_CLASSES = ["negative", "positive"]

# Pick the first review (change index for others, or use random.sample for variety)
PT_SAMPLE_TEXT = imdb_df["review"].iloc[0]

print("Sample review snippet:\n", PT_SAMPLE_TEXT[:400], "...")

print("Running LLM summary + zero-shot on an IMDB review…")
summary = summarize(PT_SAMPLE_TEXT)
...





Token indices sequence length is longer than the specified maximum sequence length for this model (616 > 512). Running this sequence through the model will result in indexing errors


Sample review snippet:
 With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film whic ...
Running LLM summary + zero-shot on an IMDB review…


Ellipsis

In [10]:
# Re-run this after kernel restart to see a compact end-to-end run:
# (It assumes you already executed the previous cells once.)
print("✔ PyTorch text classifier trained & evaluated")
print("✔ TensorFlow tabular model trained & evaluated")
print("✔ LLM summary + zero-shot label completed")


✔ PyTorch text classifier trained & evaluated
✔ TensorFlow tabular model trained & evaluated
✔ LLM summary + zero-shot label completed
